
# Supply Chain Network Optimization

This notebook demonstrates the optimization of a supply chain network using Python and the PuLP library. We will:
- Import the necessary data.
- Define the problem and objective function.
- Add constraints for factories, distribution centers, and customers.
- Solve the optimization problem and analyze the results.



## Step 1: Import Necessary Libraries

We will use the following libraries:
- **pandas**: For data manipulation.
- **pulp**: For solving linear programming problems.


In [1]:
pip install nbformat



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pulp

  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 1.0 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger



## Step 2: Load and Prepare Data

Load the distance matrix, factories, distribution centers, and customer demand data.


In [14]:

# Set working directory and read data (adjust the path as needed)
dist = pd.read_csv("Distance Matrix.csv")

# Add index column for reference
dist['Index'] = range(1, len(dist) + 1)

# Define factories and their capacities
factories = pd.DataFrame({'Factory': dist['Source'].unique()[:2], 'Capacity': [150000, 200000]})

# Define distribution centers and their throughput capacities
distribution_centers = pd.DataFrame({
    'DistCentre': dist['Source'].unique()[2:6],
    'Throughput': [70000, 50000, 100000, 40000]
})

# Define customers and their demand
customers = pd.DataFrame({
    'Customer': dist['Destination'].unique()[4:10],
    'Demand': [50000, 10000, 40000, 35000, 60000, 20000]
})

# Print the data for verification
print(dist)



        Source Destination  Distance  Index
0    Liverpool   Newcastle       178      1
1    Liverpool  Birmingham        99      2
2    Liverpool      London       221      3
3    Liverpool      Exeter       251      4
4     Brighton   Newcastle       345      5
5     Brighton  Birmingham       176      6
6     Brighton      London        65      7
7     Brighton      Exeter       206      8
8    Newcastle    Carlisle        60      9
9    Newcastle  Darlington        37     10
10   Newcastle   Sheffield       130     11
11   Newcastle   Cambridge       236     12
12   Newcastle      Oxford       258     13
13   Newcastle       Truro       458     14
14  Birmingham    Carlisle       193     15
15  Birmingham  Darlington       172     16
16  Birmingham   Sheffield        91     17
17  Birmingham   Cambridge        99     18
18  Birmingham      Oxford        79     19
19  Birmingham       Truro       249     20
20      London    Carlisle       307     21
21      London  Darlington      

In [15]:
print(factories)


     Factory  Capacity
0  Liverpool    150000
1   Brighton    200000


In [17]:
print(distribution_centers)

   DistCentre  Throughput
0   Newcastle       70000
1  Birmingham       50000
2      London      100000
3      Exeter       40000


In [16]:
print(customers)

     Customer  Demand
0    Carlisle   50000
1  Darlington   10000
2   Sheffield   40000
3   Cambridge   35000
4      Oxford   60000
5       Truro   20000



## Step 3: Define the Optimization Problem

We will define the optimization problem using PuLP, specifying the objective function (minimizing cost) and the constraints.


In [10]:

# Create LP problem
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Create decision variables for quantities to be shipped
decision_vars = LpVariable.dicts("Qty", (dist['Index']), lowBound=0, cat=LpInteger)

# Objective function: Minimize cost = quantity shipped x distance
model += lpSum([decision_vars[i] * dist.loc[i - 1, 'Distance'] for i in dist['Index']])



## Step 4: Add Constraints

We will add constraints for:
- Factory capacities
- Distribution centers (flow balance and throughput limits)
- Customer demand


In [11]:

# Capacity constraints for factories
for factory in factories['Factory']:
    factory_data = dist[dist['Source'] == factory]
    model += lpSum([decision_vars[i] for i in factory_data['Index']]) <= factories.loc[factories['Factory'] == factory, 'Capacity'].values[0]

# Constraints for distribution centers: Qty out of DC - Qty into DC = 0, Qty out of DC <= throughput
for dc in distribution_centers['DistCentre']:
    dc_out_data = dist[dist['Source'] == dc]
    dc_in_data = dist[dist['Destination'] == dc]
    
    model += lpSum([decision_vars[i] for i in dc_out_data['Index']]) - lpSum([decision_vars[i] for i in dc_in_data['Index']]) == 0
    model += lpSum([decision_vars[i] for i in dc_out_data['Index']]) <= distribution_centers.loc[distribution_centers['DistCentre'] == dc, 'Throughput'].values[0]

# Constraints for customer demands: Quantity into customer = demand
for customer in customers['Customer']:
    customer_data = dist[dist['Destination'] == customer]
    model += lpSum([decision_vars[i] for i in customer_data['Index']]) == customers.loc[customers['Customer'] == customer, 'Demand'].values[0]



## Step 5: Solve the Model

Use the PuLP solver to solve the optimization problem and obtain the optimal solution.


In [18]:
# Solve the optimization problem using PuLP's solve method
model.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/suyashranjan/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/s2/nn8l_8pj1vv5ppnf6gr5n1pm0000gn/T/c31d7e29a9e24434b798c7477ce7d64f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/s2/nn8l_8pj1vv5ppnf6gr5n1pm0000gn/T/c31d7e29a9e24434b798c7477ce7d64f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 206 RHS
At line 223 BOUNDS
At line 256 ENDATA
Problem MODEL has 16 rows, 32 columns and 88 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.9135e+07 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 16 rows, 32 columns (32 integer (0 of which binary)) and 88 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 

1

In [19]:
# Print the optimized cost
print(f"Optimized Cost: {model.objective.value():.2f}")


Optimized Cost: 39135000.00


In [20]:
# Extract the optimized shipment quantities for each route
dist['Qty'] = [decision_vars[i].varValue for i in dist['Index']]

# Print only the routes with non-zero shipment quantities
optimized_routes = dist[dist['Qty'] > 0]

# Display the optimized shipment quantities
print("Optimized Shipment Quantities:")
print(optimized_routes)


Optimized Shipment Quantities:
        Source Destination  Distance  Index      Qty
0    Liverpool   Newcastle       178      1  60000.0
1    Liverpool  Birmingham        99      2  40000.0
6     Brighton      London        65      7  95000.0
7     Brighton      Exeter       206      8  20000.0
8    Newcastle    Carlisle        60      9  50000.0
9    Newcastle  Darlington        37     10  10000.0
16  Birmingham   Sheffield        91     17  40000.0
23      London   Cambridge        58     24  35000.0
24      London      Oxford        56     25  60000.0
31      Exeter       Truro        90     32  20000.0



## Step 6: Modify Factory Capacities and Re-run Solver

We will change the factory capacities and solve the problem again to see the impact.


In [21]:

# Modify factory capacities
factories.loc[factories['Factory'] == 'Liverpool', 'Capacity'] = 500000
factories.loc[factories['Factory'] == 'Brighton', 'Capacity'] = 100000

# Re-run solver
model.solve()

# Print the optimized cost after changing capacities
print(f"Optimized Cost (After Capacity Changes): {model.objective.value()}")

# Print the updated quantities for shipments
dist['Qty'] = [decision_vars[i].varValue for i in dist['Index']]
print(dist[dist['Qty'] > 0])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/suyashranjan/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/s2/nn8l_8pj1vv5ppnf6gr5n1pm0000gn/T/3f77f08f8acc44518086dd66dffcd6bb-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/s2/nn8l_8pj1vv5ppnf6gr5n1pm0000gn/T/3f77f08f8acc44518086dd66dffcd6bb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 206 RHS
At line 223 BOUNDS
At line 256 ENDATA
Problem MODEL has 16 rows, 32 columns and 88 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.9135e+07 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 16 rows, 32 columns (32 integer (0 of which binary)) and 88 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 


## Conclusion

This notebook demonstrates the process of optimizing a supply chain network using linear programming techniques. 
We utilized Python and the PuLP library to minimize the shipping costs while satisfying the constraints for factories, distribution centers, and customer demand.
